In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf


Using TensorFlow backend.


In [2]:
#image dimensions
img_width, img_height = 200, 200

In [3]:
train_data_dir = './data/train'
test_data_dir  = './data/test'
num_train_samples = 59850 #29925 in both classes  
num_test_samples  = 22828 #7498 in black, 15330 in death
epochs = 20 
batch_size = 32

In [4]:
if K.image_data_format() == 'channel_first':
    input_shape = (3, img_width, img_height)   #change 3 to 1 for grayscale
else:
    input_shape = (img_width, img_height, 3)

In [10]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3, preprocess_input

CLASSES = 2
    
# setup Inception v3 model
#pretrained on imagenet, include_top=False to exclude final fully connected layer (where the transfer learning is gonna take place)
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.5)(x) 
predictions = Dense(CLASSES, activation='softmax')(x) #softmax for multiclass
model = Model(inputs=base_model.input, outputs=predictions)
   
# transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [11]:
#change loss to categorical_crossentropy for multiclass
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

save_best_checkpoint = ModelCheckpoint('best_inception_weights.h5', save_best_only=True, monitor='val_loss', mode='min')

Found 59849 images belonging to 2 classes.
Found 22828 images belonging to 2 classes.


In [13]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples//batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=num_test_samples//batch_size,
    callbacks=[save_best_checkpoint])

#roughly % accuracy on greyscale
#roughly 71.6% accuracy on rgb

Epoch 1/20
1870/1870 [==============================] - 10284s 5s/step - loss: 0.6974 - acc: 0.6212 - val_loss: 0.6783 - val_acc: 0.6542
Epoch 2/20
1870/1870 [==============================] - 10352s 6s/step - loss: 0.6831 - acc: 0.6289 - val_loss: 0.6617 - val_acc: 0.6741
Epoch 3/20
1870/1870 [==============================] - 10287s 6s/step - loss: 0.6836 - acc: 0.6284 - val_loss: 0.7599 - val_acc: 0.7081
Epoch 4/20
1870/1870 [==============================] - 10311s 6s/step - loss: 0.6832 - acc: 0.6274 - val_loss: 0.6722 - val_acc: 0.6702
Epoch 5/20
1870/1870 [==============================] - 10244s 5s/step - loss: 0.6819 - acc: 0.6285 - val_loss: 0.6555 - val_acc: 0.7161
Epoch 6/20
1870/1870 [==============================] - 10272s 5s/step - loss: 0.6848 - acc: 0.6251 - val_loss: 0.8543 - val_acc: 0.6963
Epoch 7/20
1870/1870 [==============================] - 10550s 6s/step - loss: 0.6842 - acc: 0.6263 - val_loss: 1.0224 - val_acc: 0.5245
Epoch 8/20
1870/1870 [===================

KeyboardInterrupt: 

In [ ]:
model.save_weights('inception_rgb_model.h5')
model.save('inception_model.model')
#model.save_weights('inception_b&w_model.h5')